In [15]:
import pandas as pd
import os
from pathlib import Path
from compute_group_stance import compute_group_sentiment, compute_group_lexicon_sentiment
from transformers import pipeline, set_seed


In [11]:
with open("../data/usa_tweets_2019/dem_4.7M_tweets_proc.txt") as f:
    dem_tweets = f.readlines()
    
with open("../data/usa_tweets_2019/repub_4.7M_tweets_proc.txt") as f:
    repub_tweets = f.readlines()


In [51]:
df = pd.read_csv("./anes2020_pilot_prompt_probing.csv")
df.head()

,Democrat,Republican,is_repub_leading,Prompt1,Prompt2,Prompt3,Prompt4,pid
0,17.664378,77.833163,True,Donald Trump,Donald Trump is,Donald Trump is a,Donald Trump is the,fttrump1
1,81.292704,29.992821,False,Barack Obama,Barack Obama is,Barack Obama is a,Barack Obama is the,ftobama1
2,66.497423,24.401235,False,Joe Biden,Joe Biden is,Joe Biden is a,Joe Biden is the,ftbiden1
3,59.843478,20.457680,False,Elizabeth Warren,Elizabeth Warren is,Elizabeth Warren is a,Elizabeth Warren is the,ftwarren1
4,63.769828,20.503597,False,Bernie Sanders,Bernie Sanders is,Bernie Sanders is a,Bernie Sanders is the,ftsanders1


## create the combined dataset for training another two GPT-2 models

In [6]:
# with open("../data/usa_tweets_2019/combined_dem_rep_9.4M_proc.txt", "w") as out:
#     for line in dem_tweets:
#         out.write(line)
#     for line in repub_tweets:
#         out.write(line)

## baseline 1: Frequency Model

In [46]:
# construct the key words

keywords = [
        "Donald Trump",
        "Barack Obama",
        "Joe Biden",
        "Elizabeth Warren",
        "Bernie Sanders",
        "Pete Buttigieg",
        "Kamala Harris",
        "Amy Klobuchar",
        "Mike Pence",
        "Andrew Yang",
        "Nancy Pelosi",
        "Marco Rubio",
        "Alexandria Ocasio-Cortez",
        "Nikki Haley",
        "Clarence Thomas",
        "Anthony Fauci",
        "Black people",
        "White people",
        "Hispanic people",
        "Asian people",
        "illegal immigrants",
        "feminists",
        "the #MeToo movement",
        "transgender people",
        "socialists",
        "capitalists",
        "big business",
        "labor unions",
        "the Republican Party",
        "the Democratic Party"
]

loose_keywords = [
        "Trump",
        "Obama",
        "Biden",
        "Warren",
        "Sanders",
        "Buttigieg",
        "Harris",
        "Klobuchar",
        "Pence",
        "Yang",
        "Pelosi",
        "Rubio",
        "Ocasio-Cortez",
        "Haley",
        "Thomas",
        "Fauci",
        "Black people",
        "White people",
        "Hispanic",
        "Asian",
        "illegal immigrant",
        "feminist",
        "#MeToo movement",
        "transgender",
        "socialist",
        "capitalist",
        "big business",
        "labor union",
        "Republican Party",
        "Democratic Party"
]


In [47]:
def identify_tweets(tweets, keyword_list):
    ret_tweets = []
    keyword_list = [keyword.lower() for keyword in keyword_list]
    for tweet_ in tweets:
        tweet = tweet_.lower()
        for word in keyword_list:
            if word in tweet:
                ret_tweets.append(tweet_)
                break
    return ret_tweets

# len(identify_tweets(repub_tweets, ["#MeToo movement"]))

In [48]:
# df_dem_counts

In [49]:
def write_tweets(community_datafolder, community_tweets, keywords, questions):
    Path(community_datafolder).mkdir(parents=True, exist_ok=True)
    rows  =[]
    for keyword, question in zip(keywords, questions):
        tweets = identify_tweets(community_tweets, [keyword])
        filename = os.path.join(community_datafolder, question+".txt")
        print(filename)
        with open(filename, 'w') as out:
            for tweet in tweets:
                out.write(tweet)
        rows.append([keyword, question, len(tweets)])
        # print(keyword, len(tweets))
    return pd.DataFrame(rows, columns=['keyword', 'question', 'tweet_num'])

In [59]:
# use the exact match with the FULL NAMES
# df_dem_counts = write_tweets("../output/exact_keyword_2019_dem/", dem_tweets, keywords, df.pid.values)

# df_repub_counts = write_tweets("../output/exact_keyword_2019_repub/", repub_tweets, keywords, df.pid.values)


def count_keywords(community_datafolder, keywords, questions):
    rows  =[]
    for keyword, question in zip(keywords, questions):
        filename = os.path.join(community_datafolder, question+".txt")
        with open(filename) as f:
            num_tweets = len(f.readlines())
            rows.append([keyword, question, num_tweets])
    return pd.DataFrame(rows, columns=['keyword', 'question', 'tweet_num']).sort_values(by=['question'])

count_keywords("../output/loose_keyword_2019_repub/", loose_keywords, df.pid.values)

,keyword,question,tweet_num
19,Asian,ftasian,1917
2,Biden,ftbiden1,21748
26,big business,ftbigbusiness,291
16,Black people,ftblack,1278
5,Buttigieg,ftbuttigieg1,1348
25,capitalist,ftcapitalists,941
29,Democratic Party,ftdemocraticparty,3611
15,Fauci,ftfauci1,1219
21,feminist,ftfeminists,1470
13,Haley,fthaley1,712


In [119]:
def compute_keyword_model_accuracy(df_anes, df_dem, df_repub):
    df_list = []
    df_repub['prediction'] = (df_repub['group_sentiment'] < df_dem['group_sentiment'])

    gold_labels = df_anes.is_repub_leading.astype(int).values
    pred_labels = df_repub.prediction.astype(int).values
    acc = accuracy_score(gold_labels, pred_labels)
    return acc

df_repub_counts['group_sentiment'] = df_repub_counts.tweet_num
df_dem_counts['group_sentiment'] = df_dem_counts.tweet_num
df = pd.read_csv("./anes2020_pilot_prompt_probing.csv")

compute_keyword_model_accuracy(df, df_dem_counts, df_repub_counts)

# df_repub_counts.tweet_num - df_dem_counts.tweet_num

0.5333333333333333

In [43]:
# use the loose match with the FULL NAMES
df_dem_counts = write_tweets("../output/loose_keyword_2019_dem/", dem_tweets, loose_keywords, df.pid.values)

df_repub_counts = write_tweets("../output/loose_keyword_2019_repub/", repub_tweets, loose_keywords, df.pid.values)

NameError: name 'write_tweets' is not defined

In [42]:
df_freq_counts = df_repub_counts[['keyword', 'question']]
df_freq_counts['loose_count_democrat'] = df_dem_counts['tweet_num']
df_freq_counts['loose_count_republican'] = df_repub_counts['tweet_num']
df_freq_counts

NameError: name 'df_repub_counts' is not defined

In [20]:
def compute_keyword_model_accuracy(df_anes, df_dem, df_repub):
    df_list = []
    df_repub['prediction'] = (df_repub['group_sentiment'] > df_dem['group_sentiment'])

    gold_labels = df_anes.is_repub_leading.astype(int).values
    pred_labels = df_repub.prediction.astype(int).values
    acc = accuracy_score(gold_labels, pred_labels)
    return acc

df_repub_counts['group_sentiment'] = df_repub_counts.tweet_num
df_dem_counts['group_sentiment'] = df_dem_counts.tweet_num
df = pd.read_csv("./anes2020_pilot_prompt_probing.csv")

compute_keyword_model_accuracy(df, df_dem_counts, df_repub_counts)

# df_repub_counts.tweet_num - df_dem_counts.tweet_num

0.4666666666666667

## baseline 2: Keyword Retrieval

In [19]:
from sklearn.metrics import accuracy_score

def compute_keyword_model_accuracy(df_anes, df_dem, df_repub):
    df_list = []
    df_repub['prediction'] = (df_repub['group_sentiment'] > df_dem['group_sentiment'])

    gold_labels = df_anes.is_repub_leading.astype(int).values
    pred_labels = df_repub.prediction.astype(int).values
    acc = accuracy_score(gold_labels, pred_labels)
    return acc

In [20]:
# exact_keyword + lexicon
df_dem = pd.read_csv("../output/exact_keyword_2019_dem/group_stance_lexicon_predictions.csv")
df_repub = pd.read_csv("../output/exact_keyword_2019_repub/group_stance_lexicon_predictions.csv")
df = pd.read_csv("./anes2020_pilot_prompt_probing.csv")
df_scores = compute_keyword_model_accuracy(df, df_dem, df_repub)
df_scores

0.7666666666666667

In [43]:
# extract gold ranks
df_politician_results = df.sort_values(by=["pid"])
gold_dem_rank = df['Democrat'].rank()
gold_repub_rank = df['Republican'].rank()
gold_repub_rank

from scipy import stats
def extract_ranking(df_):
    df_ = df_.sort_values(by=['question'])
    return df_.groupby(['question']).group_sentiment.mean().rank()

dem_rank = extract_ranking(df_dem)
repub_rank = extract_ranking(df_repub)

tau, p_value = stats.kendalltau(dem_rank, gold_dem_rank)
print("Democratic", tau, p_value)

tau, p_value = stats.kendalltau(repub_rank, gold_repub_rank)
print("Republican", tau, p_value)

Democratic 0.19080459770114944 0.1445086040145618
Republican -0.22758620689655176 0.08031212415343415


In [40]:
# exact_keyword + neural
df_dem = pd.read_csv("../output/exact_keyword_2019_dem/group_stance_neural_predictions.csv")
df_repub = pd.read_csv("../output/exact_keyword_2019_repub/group_stance_neural_predictions.csv")
df = pd.read_csv("./anes2020_pilot_prompt_probing.csv")
df_scores = compute_keyword_model_accuracy(df, df_dem, df_repub)
df_scores

0.8666666666666667

In [41]:
## error analysis
df_error = df[['Prompt1']]
df_error['gold_label'] = df.is_repub_leading.astype(int)
df_error['pred_label'] = (df_repub['group_sentiment'] > df_dem['group_sentiment']).astype(int)
df_error[df_error.gold_label != df_error.pred_label]

/tmp/ipykernel_33264/3819427974.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_error['gold_label'] = df.is_repub_leading.astype(int)
/tmp/ipykernel_33264/3819427974.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_error['pred_label'] = (df_repub['group_sentiment'] > df_dem['group_sentiment']).astype(int)


,Prompt1,gold_label,pred_label
9,Andrew Yang,0,1
17,White people,1,0
20,illegal immigrants,0,1
23,transgender people,0,1


In [56]:
dem_rank = extract_ranking(df_dem)
repub_rank = extract_ranking(df_repub)

tau, p_value = stats.kendalltau(dem_rank, gold_dem_rank)
print("Democratic", tau, p_value)

tau, p_value = stats.kendalltau(repub_rank, gold_repub_rank)
print("Republican", tau, p_value)

Democratic -0.006896551724137931 0.9717917196756787
Republican -0.09885057471264368 0.45698072495792497


array([ 1., 30., 23., 16., 22., 13., 12., 10.,  2.,  9., 18.,  6., 11.,
        4.,  5., 24., 29., 27., 28., 25., 15., 19., 21., 20., 14.,  8.,
        7., 17.,  3., 26.])

In [25]:
# loose_keyword + lexicon
df_dem = pd.read_csv("../output/loose_keyword_2019_dem/group_stance_lexicon_predictions.csv")
df_repub = pd.read_csv("../output/loose_keyword_2019_repub/group_stance_lexicon_predictions.csv")
df = pd.read_csv("./anes2020_pilot_prompt_probing.csv")
df_scores = compute_keyword_model_accuracy(df, df_dem, df_repub)
df_scores

0.9333333333333333

In [39]:
## error analysis
df_error = df[['Prompt1']]
df_error['gold_label'] = df.is_repub_leading.astype(int)
df_error['pred_label'] = (df_repub['group_sentiment'] > df_dem['group_sentiment']).astype(int)
df_error[df_error.gold_label != df_error.pred_label]

/tmp/ipykernel_33264/3819427974.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_error['gold_label'] = df.is_repub_leading.astype(int)
/tmp/ipykernel_33264/3819427974.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_error['pred_label'] = (df_repub['group_sentiment'] > df_dem['group_sentiment']).astype(int)


,Prompt1,gold_label,pred_label
20,illegal immigrants,0,1
26,big business,1,0


In [50]:
dem_rank = extract_ranking(df_dem)
repub_rank = extract_ranking(df_repub)

tau, p_value = stats.kendalltau(dem_rank, gold_dem_rank)
print("Democratic", tau, p_value)

tau, p_value = stats.kendalltau(repub_rank, gold_repub_rank)
print("Republican", tau, p_value)

Democratic -0.10344827586206898 0.43575827915457105
Republican -0.3517241379310345 0.005952597596458611


In [51]:
# loose_keyword + lexicon
df_dem = pd.read_csv("../output/loose_keyword_2019_dem/group_stance_neural_predictions.csv")
df_repub = pd.read_csv("../output/loose_keyword_2019_repub/group_stance_neural_predictions.csv")
df = pd.read_csv("./anes2020_pilot_prompt_probing.csv")
df_scores = compute_keyword_model_accuracy(df, df_dem, df_repub)
df_scores

0.9333333333333333

In [52]:
dem_rank = extract_ranking(df_dem)
repub_rank = extract_ranking(df_repub)

tau, p_value = stats.kendalltau(dem_rank, gold_dem_rank)
print("Democratic", tau, p_value)

tau, p_value = stats.kendalltau(repub_rank, gold_repub_rank)
print("Republican", tau, p_value)

Democratic -0.006896551724137931 0.9717917196756787
Republican -0.09885057471264368 0.45698072495792497


## GPT-3 results

In [43]:
sentiment_pipeline = pipeline("sentiment-analysis",
                            model="cardiffnlp/twitter-roberta-base-sentiment-latest",
                            tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest",
                            device=0)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [94]:
columns = ['model_name', 'run', 'prompt_format', 'question', 'group_sentiment']
sentiment_model_type = 'neural'
questions = pd.read_csv("./anes2020_pilot_prompt_probing.csv").pid.values.tolist()

model_name = 'democrat-gpt3'
data_folder = '../output/{}'.format(model_name)
rows = []
for run in range(1, 2):
    run = "run_{}".format(run)
    print("Processing {} ...".format(run))
    for prompt_format in range(1, 5):
        prompt_format = "Prompt{}".format(prompt_format)
        for question in questions:
            file_name = os.path.join(data_folder, run, prompt_format, question+".txt")
            if sentiment_model_type == "neural":
                group_sentiment = compute_group_sentiment(file_name, sentiment_pipeline)
            else:
                group_sentiment = compute_group_lexicon_sentiment(file_name, sentiment_pipeline)
            rows.append([model_name, run, prompt_format, question, group_sentiment])

columns = ['model_name', 'run', 'prompt_format', 'question', 'group_sentiment']
df_gpt3_dem_scores = pd.DataFrame(rows, columns=columns)
df_gpt3_dem_scores

Processing run_1 ...


/home/hjian42/.conda/envs/change/lib/python3.8/site-packages/transformers/pipelines/base.py:997: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,model_name,run,prompt_format,question,group_sentiment
0,democrat-gpt3,run_1,Prompt1,fttrump1,36.500000
1,democrat-gpt3,run_1,Prompt1,ftobama1,30.693069
2,democrat-gpt3,run_1,Prompt1,ftbiden1,27.000000
3,democrat-gpt3,run_1,Prompt1,ftwarren1,29.500000
4,democrat-gpt3,run_1,Prompt1,ftsanders1,42.000000
...,...,...,...,...,...
115,democrat-gpt3,run_1,Prompt4,ftcapitalists,16.500000
116,democrat-gpt3,run_1,Prompt4,ftbigbusiness,13.000000
117,democrat-gpt3,run_1,Prompt4,ftlaborunions,14.000000
118,democrat-gpt3,run_1,Prompt4,ftrepublicanparty,26.000000


In [95]:
model_name = 'republican-gpt3'
rows = []
data_folder = '../output/{}'.format(model_name)
for run in range(1, 2):
    run = "run_{}".format(run)
    print("Processing {} ...".format(run))
    for prompt_format in range(1, 5):
        prompt_format = "Prompt{}".format(prompt_format)
        for question in questions:
            file_name = os.path.join(data_folder, run, prompt_format, question+".txt")
            if sentiment_model_type == "neural":
                group_sentiment = compute_group_sentiment(file_name, sentiment_pipeline)
            else:
                group_sentiment = compute_group_lexicon_sentiment(file_name, sentiment_pipeline)
            rows.append([model_name, run, prompt_format, question, group_sentiment])

columns = ['model_name', 'run', 'prompt_format', 'question', 'group_sentiment']
df_gpt3_repub_scores = pd.DataFrame(rows, columns=columns)
df_gpt3_repub_scores

Processing run_1 ...


/home/hjian42/.conda/envs/change/lib/python3.8/site-packages/transformers/pipelines/base.py:997: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,model_name,run,prompt_format,question,group_sentiment
0,republican-gpt3,run_1,Prompt1,fttrump1,38.500000
1,republican-gpt3,run_1,Prompt1,ftobama1,28.000000
2,republican-gpt3,run_1,Prompt1,ftbiden1,30.500000
3,republican-gpt3,run_1,Prompt1,ftwarren1,31.683168
4,republican-gpt3,run_1,Prompt1,ftsanders1,38.500000
...,...,...,...,...,...
115,republican-gpt3,run_1,Prompt4,ftcapitalists,25.225225
116,republican-gpt3,run_1,Prompt4,ftbigbusiness,20.000000
117,republican-gpt3,run_1,Prompt4,ftlaborunions,17.647059
118,republican-gpt3,run_1,Prompt4,ftrepublicanparty,21.500000


In [96]:
from sklearn.metrics import accuracy_score

def compute_scores(df_anes, df_dem, df_repub):
    df_list = []
    df_repub['prediction'] = (df_repub['group_sentiment'] > df_dem['group_sentiment'])

    gold_labels = df_anes.is_repub_leading.astype(int).values
    rows = []
    for run in range(1, 2):
        run = "run_{}".format(run)
        for prompt_format in range(1, 5):
            prompt_format = "Prompt{}".format(prompt_format)
            df_ = df_repub[(df_repub.run == run) & (df_repub.prompt_format == prompt_format)]
            pred_labels = df_.prediction.astype(int).values
            acc = accuracy_score(gold_labels, pred_labels)
            rows.append([run, prompt_format, acc])
    df_scores = pd.DataFrame(rows, columns=["run", "prompt_format", "accuracy"])
    return df_scores

In [97]:
compute_scores(df, df_gpt3_dem_scores, df_gpt3_repub_scores)

,run,prompt_format,accuracy
0,run_1,Prompt1,0.466667
1,run_1,Prompt2,0.600000
2,run_1,Prompt3,0.500000
3,run_1,Prompt4,0.500000


In [92]:
df_gpt3_dem_scores

,model_name,run,prompt_format,question,group_sentiment
0,democrat-gpt3,run_1,Prompt1,fttrump1,38.500000
1,democrat-gpt3,run_1,Prompt1,ftobama1,28.000000
2,democrat-gpt3,run_1,Prompt1,ftbiden1,30.500000
3,democrat-gpt3,run_1,Prompt1,ftwarren1,31.683168
4,democrat-gpt3,run_1,Prompt1,ftsanders1,38.500000
...,...,...,...,...,...
115,democrat-gpt3,run_1,Prompt4,ftcapitalists,25.225225
116,democrat-gpt3,run_1,Prompt4,ftbigbusiness,20.000000
117,democrat-gpt3,run_1,Prompt4,ftlaborunions,17.647059
118,democrat-gpt3,run_1,Prompt4,ftrepublicanparty,21.500000


In [93]:
df_gpt3_repub_scores

,model_name,run,prompt_format,question,group_sentiment,prediction
0,republican-gpt3,run_1,Prompt1,fttrump1,38.500000,False
1,republican-gpt3,run_1,Prompt1,ftobama1,28.000000,False
2,republican-gpt3,run_1,Prompt1,ftbiden1,30.500000,False
3,republican-gpt3,run_1,Prompt1,ftwarren1,31.683168,False
4,republican-gpt3,run_1,Prompt1,ftsanders1,38.500000,False
...,...,...,...,...,...,...
115,republican-gpt3,run_1,Prompt4,ftcapitalists,25.225225,False
116,republican-gpt3,run_1,Prompt4,ftbigbusiness,20.000000,False
117,republican-gpt3,run_1,Prompt4,ftlaborunions,17.647059,False
118,republican-gpt3,run_1,Prompt4,ftrepublicanparty,21.500000,False


## pre-trained GPT-2

In [4]:
from sklearn.metrics import accuracy_score
import pandas as pd

def compute_scores(df_anes, df_dem, df_repub):
    df_list = []
    df_repub['prediction'] = (df_repub['group_sentiment'] > df_dem['group_sentiment'])

    gold_labels = df_anes.is_repub_leading.astype(int).values
    rows = []
    for run in range(1, 6):
        run = "run_{}".format(run)
        for prompt_format in range(1, 5):
            prompt_format = "Prompt{}".format(prompt_format)
            df_ = df_repub[(df_repub.run == run) & (df_repub.prompt_format == prompt_format)]
            pred_labels = df_.prediction.astype(int).values
            acc = accuracy_score(gold_labels, pred_labels)
            rows.append([run, prompt_format, acc])
    df_scores = pd.DataFrame(rows, columns=["run", "prompt_format", "accuracy"])
    return df_scores

In [7]:
df = pd.read_csv("anes2020_pilot_prompt_probing.csv")
df_dem = pd.read_csv("../output/gpt2_dem/group_stance_predictions.csv")
df_repub = pd.read_csv("../output/gpt2_repub/group_stance_predictions.csv")
df_scores = compute_scores(df, df_dem, df_repub)
df_scores

,run,prompt_format,accuracy
0,run_1,Prompt1,0.700000
1,run_1,Prompt2,0.733333
2,run_1,Prompt3,0.766667
3,run_1,Prompt4,0.766667
4,run_2,Prompt1,0.733333
5,run_2,Prompt2,0.700000
6,run_2,Prompt3,0.766667
7,run_2,Prompt4,0.733333
8,run_3,Prompt1,0.766667
9,run_3,Prompt2,0.733333


In [8]:
df = pd.read_csv("anes2020_pilot_prompt_probing.csv")
df_dem = pd.read_csv("../output/gpt2_dem/group_stance_lexicon_predictions.csv")
df_repub = pd.read_csv("../output/gpt2_repub/group_stance_lexicon_predictions.csv")
df_scores = compute_scores(df, df_dem, df_repub)
df_scores

,run,prompt_format,accuracy
0,run_1,Prompt1,0.766667
1,run_1,Prompt2,0.766667
2,run_1,Prompt3,0.766667
3,run_1,Prompt4,0.766667
4,run_2,Prompt1,0.766667
5,run_2,Prompt2,0.766667
6,run_2,Prompt3,0.766667
7,run_2,Prompt4,0.666667
8,run_3,Prompt1,0.766667
9,run_3,Prompt2,0.766667


## Baseline trained GPT-2 (combined)

In [9]:
df = pd.read_csv("anes2020_pilot_prompt_probing.csv")
df_dem = pd.read_csv("../output/baseline_trained_gpt2_dem/group_stance_predictions.csv")
df_repub = pd.read_csv("../output/baseline_trained_gpt2_repub/group_stance_predictions.csv")
df_scores = compute_scores(df, df_dem, df_repub)
df_scores

,run,prompt_format,accuracy
0,run_1,Prompt1,0.533333
1,run_1,Prompt2,0.466667
2,run_1,Prompt3,0.500000
3,run_1,Prompt4,0.500000
4,run_2,Prompt1,0.566667
5,run_2,Prompt2,0.433333
6,run_2,Prompt3,0.433333
7,run_2,Prompt4,0.433333
8,run_3,Prompt1,0.433333
9,run_3,Prompt2,0.566667


In [10]:
df = pd.read_csv("anes2020_pilot_prompt_probing.csv")
df_dem = pd.read_csv("../output/baseline_trained_gpt2_dem/group_stance_lexicon_predictions.csv")
df_repub = pd.read_csv("../output/baseline_trained_gpt2_repub/group_stance_lexicon_predictions.csv")
df_scores = compute_scores(df, df_dem, df_repub)
df_scores

,run,prompt_format,accuracy
0,run_1,Prompt1,0.466667
1,run_1,Prompt2,0.666667
2,run_1,Prompt3,0.533333
3,run_1,Prompt4,0.533333
4,run_2,Prompt1,0.600000
5,run_2,Prompt2,0.433333
6,run_2,Prompt3,0.400000
7,run_2,Prompt4,0.633333
8,run_3,Prompt1,0.533333
9,run_3,Prompt2,0.500000


## Baseline fine-tuned GPT-2 (comebined)

In [13]:
df = pd.read_csv("anes2020_pilot_prompt_probing.csv")
df_dem = pd.read_csv("../output/baseline_finetuned_gpt2_dem/group_stance_predictions.csv")
df_repub = pd.read_csv("../output/baseline_finetuned_gpt2_repub/group_stance_predictions.csv")
df_scores = compute_scores(df, df_dem, df_repub)
df_scores

,run,prompt_format,accuracy
0,run_1,Prompt1,0.566667
1,run_1,Prompt2,0.566667
2,run_1,Prompt3,0.566667
3,run_1,Prompt4,0.500000
4,run_2,Prompt1,0.566667
5,run_2,Prompt2,0.500000
6,run_2,Prompt3,0.533333
7,run_2,Prompt4,0.266667
8,run_3,Prompt1,0.466667
9,run_3,Prompt2,0.466667


In [14]:
df = pd.read_csv("anes2020_pilot_prompt_probing.csv")
df_dem = pd.read_csv("../output/baseline_finetuned_gpt2_dem/group_stance_lexicon_predictions.csv")
df_repub = pd.read_csv("../output/baseline_finetuned_gpt2_repub/group_stance_lexicon_predictions.csv")
df_scores = compute_scores(df, df_dem, df_repub)
df_scores

,run,prompt_format,accuracy
0,run_1,Prompt1,0.500000
1,run_1,Prompt2,0.500000
2,run_1,Prompt3,0.533333
3,run_1,Prompt4,0.533333
4,run_2,Prompt1,0.566667
5,run_2,Prompt2,0.533333
6,run_2,Prompt3,0.466667
7,run_2,Prompt4,0.500000
8,run_3,Prompt1,0.466667
9,run_3,Prompt2,0.533333
